Nhóm 7:
    21120083_Phan Văn Khải
    21120096_Hồ Châu Long
    21120126_Nguyễn Tấn Hoàng Sa

=============================================Thu Thập Dữ Liệu===================================

In [1]:
import numpy as np
import selenium
from selenium import webdriver
from time import sleep
import random
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from selenium.webdriver.common.by import By
import pandas as pd
import os

2.1.1 Thu thập dữ liệu

In [ ]:
driver = selenium.webdriver.Chrome()
base_url = "https://ourworldindata.org/explorers/energy?"

In [2]:
def crawl_data(year, type):
    url = base_url+'tab=table&time={y}&facet=none&country=USA~GBR~CHN~OWID_WRL~IND~BRA~ZAF&hideControls=false&Total+or+Breakdown=Select+a+source&Energy+or+Electricity=Primary+energy&Metric=Annual+consumption&Select+a+source={t}'.format(y = year, t = type)
    driver.get(url)
    sleep(random.randint(5,10))

    elems_coutry = driver.find_elements(By.CSS_SELECTOR, ".entity")
    country = [elem.text for elem in elems_coutry]

    data =[]
    data.append(0)
    nam = []
    for i in range (1,len(country)+1):
        nam.append(int(year))
        try:
            elem_data = driver.find_element("xpath", "/html/body/main/div/div[3]/div/div[4]/div/div[2]/table/tbody/tr[{}]/td[2]".format(i))
            data.append(elem_data.text)
        except NoSuchElementException:
            data.append(None)

    df = pd.DataFrame(list(zip(country,nam,data)),columns= ['Country','Year',type])
    df = df = df.drop([0], axis=0)
    df = df.drop(labels=df.index[-24:])
    return df

In [3]:
def crawl_type_data(type):
    df = pd.DataFrame(columns= ['Country','Year',type])
    for i in range(2000,2023):
        df1 = crawl_data(i, type)
        df = pd.merge(df, df1 ,how='outer',on=['Country','Year',type])
    return df

In [4]:

def crawl_full_data(types):
    df = pd.DataFrame(columns= ['Country','Year'])
    for type in types:
        df1 = crawl_type_data(type)
        df = pd.merge(df, df1 ,how='outer',on=['Country','Year'])
    return df

In [ ]:
types = ['Fossil fuels','Coal','Oil','Gas','Renewables','Hydropower','Solar','Wind']
df = crawl_full_data(types)
df = df.set_index("Country")
folder_data = "../Data/"
pathData = os.path.join(folder_data, "data.csv")
df.to_csv(pathData)